# Severity of storms 

### 1.1 Business problem


Our project will focus on analyzing and predicting the severity of tornadoes across various regions in America in terms of property damage. To analyze this, we will take the last 10 years of tornado data to represent an approximate measure of it. To calculate this, we will establish a threshold for measuring the severity using the amount of property damage caused by tornadoes. We will predict the property damage of tornadoes and use a scale of low, medium, and high to represent the severity.  
In addition, we will also compare attitudes across each of the 4 seasons - summer, spring, winter, fall, and see if there are any trends present across the 10 year span that shows when tornadoes are most common and damaging. The trends will help us be more accurate in predicting the property damage caused by the tornadoes  per region during different seasons/times of the year.


### 1.2 Datasets


Our dataset details instances of severe weather across a 15 year period. We will be looking at Tornados across this time period. We are given the locations that the Tornados are in, the category of Tornadoes, fatalities, property damage, and the length of the storm. From this we created a target feature called severity which assigns tornados a number from 1 to 10 based on how severe they are.


### 1.3 Proposed analytics solution

 How we get to the target variable -- severity index
The severity index will be calculated by taking into account the property damage, the lives lost in the event, 


In [2]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Read all data into one single dataframe
df_all_data = pd.read_csv('./dataset/storm_event_details_2010.csv')


for i in range(2011,2021):
    df_temp = pd.read_csv(f'./dataset/storm_event_details_{i}.csv')
    df_all_data = df_all_data.append(df_temp, ignore_index=True)



/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_85206/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_data = df_all_data.append(df_temp, ignore_index=True)
/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_85206/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_data = df_all_data.append(df_temp, ignore_index=True)
/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_85206/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_data = df_all_data.append(df_temp, ignore_index=True)
/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_85206/3914789818.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [3]:
## Remove unused columns and format continuous columns

df_hur = df_all_data[df_all_data['EVENT_TYPE']=='Tornado']
df_hur = df_hur.drop(columns=['TOR_OTHER_WFO', 'END_YEARMONTH', 'EVENT_TYPE', 'END_DATE_TIME',
                                           'TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','DATA_SOURCE','EPISODE_NARRATIVE',
                                            'EVENT_NARRATIVE','WFO','SOURCE','CZ_TIMEZONE','BEGIN_AZIMUTH','END_AZIMUTH','BEGIN_LAT',
                                            'END_LAT','BEGIN_LON','END_LON','STATE_FIPS','BEGIN_RANGE','END_RANGE','DAMAGE_CROPS',
                                            'BEGIN_TIME','END_TIME','BEGIN_LOCATION','END_LOCATION','FLOOD_CAUSE','MAGNITUDE_TYPE',
                                            'MAGNITUDE','CZ_FIPS','CZ_TYPE','CZ_NAME','CATEGORY'])
cols = ['INJURIES_INDIRECT', 'INJURIES_DIRECT', 'DEATHS_INDIRECT', 'DEATHS_DIRECT']
df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
df_hur['TOR_AREA'] = df_hur['TOR_LENGTH']*df_hur['TOR_WIDTH']
df_hur = df_hur.drop(columns=['TOR_LENGTH', 'TOR_WIDTH'])
df_hur = df_hur.dropna()


/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_85206/2115226861.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)


In [4]:
## Format DAMAGE_PROPERTY column to be float instead of object

dmg = pd.DataFrame(df_hur['DAMAGE_PROPERTY'])

print(dmg)
for index, row in dmg.iterrows():
    val = row['DAMAGE_PROPERTY']
    if val[-1:] == 'B':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1000000
        # print(float(row['DAMAGE_PROPERTY'][:-1])*1000000)
    elif val[-1:] == 'M':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1000
        # print(float(row['DAMAGE_PROPERTY'][:-1])*1000)
    elif val[-1:] == 'K':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1
        # print(float(row['DAMAGE_PROPERTY'][:-1]))

df_hur['DAMAGE_PROPERTY'] = dmg

       DAMAGE_PROPERTY
75               5.00K
304              1.50M
617             10.00K
731            750.00K
732             10.00K
...                ...
688471           0.00K
688472         250.00K
688495          60.00K
688497           0.00K
688560           0.00K

[12945 rows x 1 columns]


In [5]:
## Sort by priority variable and find data split percentages

df_hur = df_hur.sort_values('DAMAGE_PROPERTY', ascending=False)
df_hur.info()
df_hur = df_hur.loc[df_hur['TOR_F_SCALE']!='EFU']
print(df_hur['TOR_F_SCALE'].value_counts()/len(df_hur) * 100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12945 entries, 100619 to 688560
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  12945 non-null  int64  
 1   BEGIN_DAY        12945 non-null  int64  
 2   END_DAY          12945 non-null  int64  
 3   EPISODE_ID       12945 non-null  int64  
 4   EVENT_ID         12945 non-null  int64  
 5   STATE            12945 non-null  object 
 6   YEAR             12945 non-null  int64  
 7   MONTH_NAME       12945 non-null  object 
 8   BEGIN_DATE_TIME  12945 non-null  object 
 9   DAMAGE_PROPERTY  12945 non-null  object 
 10  TOR_F_SCALE      12945 non-null  object 
 11  HARM_TOTAL       12945 non-null  int64  
 12  TOR_AREA         12945 non-null  float64
dtypes: float64(1), int64(7), object(5)
memory usage: 1.4+ MB
EF0    51.117296
EF1    35.737575
EF2     9.852883
EF3     2.576541
EF4     0.628231
EF5     0.087475
Name: TOR_F_SCALE, dtype: float

In [6]:
df_hur['MONTH_NAME'].value_counts()/len(df_hur) * 100

April        21.677932
May          20.548708
June         12.413519
July          7.093439
March         6.727634
August        6.067594
October       5.137177
November      4.580517
January       4.413519
February      4.159046
September     3.618290
December      3.562624
Name: MONTH_NAME, dtype: float64

In [7]:
## Split data into strata and sample proportinally (stratified sampling)

sampled_df = df_hur.groupby('TOR_F_SCALE', group_keys=False).apply(lambda x: x.sample(frac=0.1))
sampled_df
# sampled_df['MONTH_NAME'].value_counts()/len(df_hur) * 100

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,BEGIN_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,HARM_TOTAL,TOR_AREA
111626,201105,25,25,50541,297760,LOUISIANA,2011,May,25-MAY-11 21:22:00,0.0,EF0,0,69.5
204917,201203,2,2,62168,374006,KENTUCKY,2012,March,02-MAR-12 14:22:00,0.0,EF0,0,12.5
205364,201201,21,21,60174,361028,GEORGIA,2012,January,21-JAN-12 16:15:00,25.0,EF0,0,313.5
615981,201905,23,23,136825,820942,ILLINOIS,2019,May,23-MAY-19 07:15:00,0.0,EF0,0,14.6
668483,202008,10,10,150363,913059,ILLINOIS,2020,August,10-AUG-20 13:37:00,0.0,EF0,0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140763,201104,27,27,50516,301943,ALABAMA,2011,April,27-APR-11 13:40:00,0.0,EF4,1,22660.0
297452,201406,16,16,84971,514044,NEBRASKA,2014,June,16-JUN-14 15:40:00,3000.0,EF4,0,5188.7
662554,202007,8,8,148227,898306,MINNESOTA,2020,July,08-JUL-20 16:08:00,100.0,EF4,0,1534.0
661920,202007,8,8,148227,898307,MINNESOTA,2020,July,08-JUL-20 16:15:00,1500.0,EF4,3,4225.0


In [8]:
cont_features = []
for (columnName, columnData) in sampled_df.iteritems():
    if type(columnData) == int or float:
        cont_features.append(columnName)
        
print(cont_features)

['BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_DAY', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'YEAR', 'MONTH_NAME', 'BEGIN_DATE_TIME', 'DAMAGE_PROPERTY', 'TOR_F_SCALE', 'HARM_TOTAL', 'TOR_AREA']


/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_85206/489640541.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for (columnName, columnData) in sampled_df.iteritems():


In [9]:
df2 = sampled_df[cont_features]
df2.drop('STATE', axis = 1, inplace = True)
df2

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,YEAR,MONTH_NAME,BEGIN_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,HARM_TOTAL,TOR_AREA
111626,201105,25,25,50541,297760,2011,May,25-MAY-11 21:22:00,0.0,EF0,0,69.5
204917,201203,2,2,62168,374006,2012,March,02-MAR-12 14:22:00,0.0,EF0,0,12.5
205364,201201,21,21,60174,361028,2012,January,21-JAN-12 16:15:00,25.0,EF0,0,313.5
615981,201905,23,23,136825,820942,2019,May,23-MAY-19 07:15:00,0.0,EF0,0,14.6
668483,202008,10,10,150363,913059,2020,August,10-AUG-20 13:37:00,0.0,EF0,0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...
140763,201104,27,27,50516,301943,2011,April,27-APR-11 13:40:00,0.0,EF4,1,22660.0
297452,201406,16,16,84971,514044,2014,June,16-JUN-14 15:40:00,3000.0,EF4,0,5188.7
662554,202007,8,8,148227,898306,2020,July,08-JUL-20 16:08:00,100.0,EF4,0,1534.0
661920,202007,8,8,148227,898307,2020,July,08-JUL-20 16:15:00,1500.0,EF4,3,4225.0


In [11]:
from sklearn import preprocessing
scaled_df = preprocessing.MinMaxScaler().fit_transform(df2)
sdf=pd.DataFrame(scaled_df, columns=df2.columns)
#sdf['price_range']=cl_df['prie_range']
sdf.head()


KeyboardInterrupt



In [2]:
# visualize damage property scale of tornado with a scattarplot and 
#tor_f_scale 

#df_hur.dropna()
#df_hur = df_hur.astype({'DAMAGE_PROPERTY':'float'})

#df_hur['logarithm_base10'] = np.log10(df_hur['DAMAGE_PROPERTY'])
#df_hur['natural_log'] = np.log(df_hur['DAMAGE_PROPERTY'])
#plt.scatter(df_hur.DAMAGE_PROPERTY, df_hur.HARM_TOTAL, linewidths = 2, edgecolor ="green", s = 1, hue = 'TOR_F_SCALE')


sns.set(rc={'figure.figsize':(12, 8)})
sns.scatterplot(data=sdf, x='DAMAGE_PROPERTY', y='HARM_TOTAL', palette='deep')
#robust scaling 


plt.xlabel('property damage', fontsize=10)
plt.ylabel('total harm', fontsize=10)
plt.xlim(0.1)
plt.show()


correlation = df_hur.corr()
print(correlation.loc['DAMAGE_PROPERTY', 'HARM_TOTAL'])


#visualize tornado area 

NameError: name 'sns' is not defined

In [12]:
ndf = sampled_df[['HARM_TOTAL', 'TOR_AREA']].copy()
ndf

,HARM_TOTAL,TOR_AREA
111626,0,69.5
204917,0,12.5
205364,0,313.5
615981,0,14.6
668483,0,79.0
...,...,...
140763,1,22660.0
297452,0,5188.7
662554,0,1534.0
661920,3,4225.0


In [15]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(max_depth = 2, criterion='entropy', random_state = 0)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(ndf, sampled_df['DAMAGE_PROPERTY'],  
                                                    test_size=0.33, random_state=0)


Y_train=Y_train.astype('int')
Y_test=Y_test.astype('int')

In [17]:
dt_clf.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=2, random_state=0)

In [18]:
y_pred = dt_clf.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, y_pred)
accuracy

0.39759036144578314

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)

cm_df = pd.DataFrame(cm)
cm_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,165,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
